In [1]:
import StreetFighter as sf

In [2]:
import numpy as np
import math
import csv
import json
from sb3_contrib import TRPO

In [3]:
kRound = 100

In [4]:

def PerforMmatch( a, b, noOfRounds, isRender ):
    # Set the state file name
    state = f'VS_{a}_{b}'
    #print( state )

    # Set model name
    modelA = TRPO.load(f'model/{a}.zip')
    modelB = TRPO.load(f'model/{b}.zip')
    
    # Create environment
    env = sf.CreateEnv( state, no_of_player=2, log_folder = None, skip=1 )
    obs = env.reset()
    
    winA = 0
    winB = 0
    
    # Loop of round
    for i in range(noOfRounds):

        # reset health 
        healthA = 176
        healthB = 176
        randomAction = 0
        sameHealhCount = 0
        
        while True:

            actions = []
            # If no attack for more then 120 frames.
            if sameHealhCount > 120:
                randomAction = 5
                sameHealhCount = 0

            # Put random action
            if randomAction > 0 :
                actionA = [env.action_space.sample()]
                actionB = [env.action_space.sample()]
                randomAction -= 1

            else:
                # get the action from model
                actionA, _ = modelA.predict(obs)
                actionB, _ = modelB.predict(obs)

            actions.append([actionA[0],actionB[0]])

            obs, rewards, done, info = env.step(actions)

            if isRender:
                env.render()
            
            # Get this frame health
            currhealthA = info[0]['health']
            currhealthB = info[0]['enemy_health']
        
            if (healthA == currhealthA) and (healthB == currhealthB) :
                sameHealhCount += 1
            
            healthA = currhealthA
            healthB = currhealthB
            
            if done == True:
                obs = env.reset()
                break
                
        # If any one win
        if info[0]['matches_won'] == 1:
            winA += 1
        elif info[0]['2p_matches_won'] == 1:
            winB += 1
        else:
            if healthA > healthB:
                winA += 1                           
            elif healthB > healthA:
                winB += 1
            else:
                winA += 0.5
                winB += 0.5
            
    return winA / kRound

In [ ]:
#PerforMmatch( 'Ryu', 'EHonda', 1, True  )

In [6]:
n_char = len(sf.kCharacters)
match_tabel = np.zeros((n_char,n_char))

# Loop each char vs each other
for i in range(n_char):
    for j in range(n_char):
        if j < i :
            continue
        if i == j:
            match_tabel[i][j] = -1
        else: 
            v = PerforMmatch( sf.kCharacters[i], sf.kCharacters[j], kRound, False ) * 10 
            print( f'{sf.kCharacters[i]} vs {sf.kCharacters[j]} > {v}')
            # put the score into match_table
            match_tabel[i][j] = v 
            match_tabel[j][i] = 10-v
            
                
        

Ryu vs Ken > 9.8
Ryu vs EHonda > 5.699999999999999
Ryu vs ChunLi > 4.75
Ryu vs Blanka > 4.25
Ryu vs Zangief > 10.0
Ryu vs Guile > 8.7
Ryu vs Dhalsim > 7.6
Ryu vs Balrog > 8.0
Ryu vs Vega > 7.4
Ryu vs Sagat > 9.399999999999999
Ryu vs MBison > 5.8999999999999995
Ken vs EHonda > 3.8
Ken vs ChunLi > 0.8
Ken vs Blanka > 2.3000000000000003
Ken vs Zangief > 9.5
Ken vs Guile > 9.3
Ken vs Dhalsim > 7.5
Ken vs Balrog > 2.0
Ken vs Vega > 3.5999999999999996
Ken vs Sagat > 1.55
Ken vs MBison > 0.0
EHonda vs ChunLi > 0.6
EHonda vs Blanka > 6.45
EHonda vs Zangief > 6.800000000000001
EHonda vs Guile > 8.100000000000001
EHonda vs Dhalsim > 9.0
EHonda vs Balrog > 7.5
EHonda vs Vega > 4.1
EHonda vs Sagat > 7.65
EHonda vs MBison > 3.5
ChunLi vs Blanka > 3.1
ChunLi vs Zangief > 9.15
ChunLi vs Guile > 10.0
ChunLi vs Dhalsim > 10.0
ChunLi vs Balrog > 9.9
ChunLi vs Vega > 9.6
ChunLi vs Sagat > 9.200000000000001
ChunLi vs MBison > 10.0
Blanka vs Zangief > 10.0
Blanka vs Guile > 6.800000000000001
Blanka vs Dhal

In [10]:
match_tabel

array([[-1.  ,  9.8 ,  5.7 ,  4.75,  4.25, 10.  ,  8.7 ,  7.6 ,  8.  ,
         7.4 ,  9.4 ,  5.9 ],
       [ 0.2 , -1.  ,  3.8 ,  0.8 ,  2.3 ,  9.5 ,  9.3 ,  7.5 ,  2.  ,
         3.6 ,  1.55,  0.  ],
       [ 4.3 ,  6.2 , -1.  ,  0.6 ,  6.45,  6.8 ,  8.1 ,  9.  ,  7.5 ,
         4.1 ,  7.65,  3.5 ],
       [ 5.25,  9.2 ,  9.4 , -1.  ,  3.1 ,  9.15, 10.  , 10.  ,  9.9 ,
         9.6 ,  9.2 , 10.  ],
       [ 5.75,  7.7 ,  3.55,  6.9 , -1.  , 10.  ,  6.8 ,  2.2 ,  5.1 ,
         6.1 ,  7.  ,  7.75],
       [ 0.  ,  0.5 ,  3.2 ,  0.85,  0.  , -1.  ,  0.6 ,  1.35,  2.95,
         0.  ,  1.45,  1.5 ],
       [ 1.3 ,  0.7 ,  1.9 ,  0.  ,  3.2 ,  9.4 , -1.  ,  0.3 ,  4.35,
         7.  ,  1.4 ,  5.6 ],
       [ 2.4 ,  2.5 ,  1.  ,  0.  ,  7.8 ,  8.65,  9.7 , -1.  , 10.  ,
         9.6 ,  9.6 ,  9.7 ],
       [ 2.  ,  8.  ,  2.5 ,  0.1 ,  4.9 ,  7.05,  5.65,  0.  , -1.  ,
         7.8 ,  0.15,  0.5 ],
       [ 2.6 ,  6.4 ,  5.9 ,  0.4 ,  3.9 , 10.  ,  3.  ,  0.4 ,  2.2 ,
        -1.  ,  9.4 

In [8]:
with open('matches.csv', 'w') as out_file:
    tsv_writer = csv.writer(out_file)    
    
    o = ['']
    o.extend(sf.kCharacters)
    tsv_writer.writerow(o)
        
    for i in range(n_char):
        l = match_tabel[i].tolist()
        o = [ sf.kCharacters[i]]
        for i in l:
            if i == -1 :
                o.append('-')
            else:
                if i % 1 == 0:
                    o.append( math.floor(i) )
                else:
                    o.append( round(i,1) )
        tsv_writer.writerow(o)
    

In [9]:
with open('../Matchup_web/data.txt', 'wt') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    tsv_writer.writerow('# Testing')
    
    o = [' ']
    o.extend(sf.kCharacters)
    tsv_writer.writerow(o)
        
    for i in range(n_char):
        l = match_tabel[i].tolist()
        o = [ sf.kCharacters[i]]
        for i in l:
            if i == -1 :
                o.append('-')
            else:
                if i % 1 == 0:
                    o.append( math.floor(i) )
                else:
                    o.append( round(i,1) )
        tsv_writer.writerow(o)
    